In [1]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# Scrape

Let's scrape data about smartphones from internet store http://rozetka.com.ua/ . We can sort products by different ways, but we interested in this two:

* sorting by rating
* sorting by popularity

In [2]:
url_rating = 'http://rozetka.com.ua/mobile-phones/c80003/page={0};preset=smartfon;view=list/'
url_popularity = 'http://rozetka.com.ua/mobile-phones/c80003/page={0};preset=smartfon;sort=popularity;view=list/'

In [3]:
page_num = 23 
pages = []
for page_number in range(1, page_num + 1):
    pages.append(url_rating.format(page_number))

print pages[0]

http://rozetka.com.ua/mobile-phones/c80003/page=1;preset=smartfon;view=list/


In [4]:
products = []

for i in range(len(pages)):
    html = urlopen(pages[i])
    soup = BeautifulSoup(html, 'html.parser')
    
    char_tab = '#tab=characteristics'
    gils = soup.find_all(class_='g-i-list-title')
    for gil in gils:
        products.append(gil.find('a', href=True)['href']+char_tab)

In [5]:
l = len(products)

In [6]:
products = pd.DataFrame(products, columns=['link'])

Items in products dataframe sorted by rating so let's add rating column. Also we add id column since we will renew this data later.

In [7]:
products['id'] = pd.Series(range(l))
products['rating'] = pd.Series(range(l))

In [8]:
products.head()

,link,id,rating
0,http://rozetka.com.ua/samsung_galaxy_j7_ds_bla...,0,0
1,http://rozetka.com.ua/huawei_y5_ii_gold_lifece...,1,1
2,http://rozetka.com.ua/samsung_sm_j320hzkdsek/p...,2,2
3,http://rozetka.com.ua/blackview_omega_pro_pw_e...,3,3
4,http://rozetka.com.ua/htc_desire_628_dual_sim_...,4,4


Now let's fetch this data sorted by popularity.

In [9]:
pages = []
for page_number in range(1, page_num + 1):
    pages.append(url_popularity.format(page_number))

In [10]:
pop_products = []
for i in range(len(pages)):
    html = urlopen(pages[i])
    soup = BeautifulSoup(html, 'html.parser')
    
    char_tab = '#tab=characteristics'
    gils = soup.find_all(class_='g-i-list-title')
    for gil in gils:
        pop_products.append(gil.find('a', href=True)['href']+char_tab)

In [11]:
pop_products = pd.DataFrame(pop_products, columns=['link'])

In [12]:
pop_products['popularity'] = pd.Series(range(l))
pop_products.head()

,link,popularity
0,http://rozetka.com.ua/meizu_m2_16gb_wh_world/p...,0
1,http://rozetka.com.ua/doogge_x5_max_pro_black/...,1
2,http://rozetka.com.ua/meizu_m3_note_32_gr_eu/p...,2
3,http://rozetka.com.ua/xiaomi_redmi_4a_2_16_gd_...,3
4,http://rozetka.com.ua/prestigio_psp7530duogrey...,4


In [13]:
data = pd.merge(products, pop_products,
               left_on=['link'], right_on=['link'])

In [14]:
data.head()

,link,id,rating,popularity
0,http://rozetka.com.ua/samsung_galaxy_j7_ds_bla...,0,0,5
1,http://rozetka.com.ua/huawei_y5_ii_gold_lifece...,1,1,11
2,http://rozetka.com.ua/samsung_sm_j320hzkdsek/p...,2,2,8
3,http://rozetka.com.ua/blackview_omega_pro_pw_e...,3,3,13
4,http://rozetka.com.ua/htc_desire_628_dual_sim_...,4,4,30


Save data:

In [16]:
import datetime

today = './data/' + str(datetime.date.today()) + '.csv'
data.to_csv(today)

We will run this kernel every two weeks to detect changes.